##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Core API를 사용하는 바이너리 분류를 위한 로지스틱 회귀

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/core/logistic_regression_core"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> TensorFlow.org에서 보기</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/core/logistic_regression_core.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a> </td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/guide/core/logistic_regression_core.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/core/logistic_regression_core.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a></td>
</table>

이 가이드에서는 [TensorFlow Core 하위 수준 API](https://www.tensorflow.org/guide/core)를 사용하여 [로지스틱 회귀](https://developers.google.com/machine-learning/crash-course/logistic-regression/){:.external}로 [바이너리 분류](https://developers.google.com/machine-learning/glossary#binary_classification){:.external}를 수행하는 방법을 보여줍니다. 이 가이드는 종양 분류를 위해 [위스콘신 유방암 데이터세트](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)){:.external}를 사용합니다.

[로지스틱 회귀](https://developers.google.com/machine-learning/crash-course/logistic-regression/){:.external}는 바이너리 분류에서 가장 널리 사용하는 알고리즘 중 하나입니다. 특성이 있는 예제 세트를 제공하 경우 로지스틱 회귀는 0과 1 사이의 값을 출력하고자 하며, 이는 특정 클래스에 속하는 각 예제의 확률로 해석할 수 있습니다. 

## 설치하기

이 튜토리얼에서는 CSV 파일을 [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html){:.external}으로 읽어오는 경우에는 [pandas](https://pandas.pydata.org){:.external}를, 데이터세트에서 쌍별 관계를 플로팅하는 경우에는 [seaborn](https://seaborn.pydata.org){:.external}을, 혼동 행렬을 계산하는 경우에는 [Scikit-learn](https://scikit-learn.org/){:.external}을, 시각화를 생성하는 경우에는 [matplotlib](https://matplotlib.org/){:.external}을 사용합니다.

In [ ]:
!pip install -q seaborn

In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn.metrics as sk_metrics
import tempfile
import os

# Preset matplotlib figure sizes.
matplotlib.rcParams['figure.figsize'] = [9, 6]

print(tf.__version__)
# To make the results reproducible, set the random seed value.
tf.random.set_seed(22)

## 데이터 로드하기

그런 다음에는 [UCI 머신러닝 리포지토리](https://archive.ics.uci.edu/ml/){:.external}의 [위스콘신 유방암 데이터세트](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)){:.external}를 로드합니다. 이 데이터세트에는 종양의 반경, 질감 및 오목한 정도와 같은 다양한 특성이 포함되어 있습니다.

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data'

features = ['radius', 'texture', 'perimeter', 'area', 'smoothness', 'compactness',
            'concavity', 'concave_poinits', 'symmetry', 'fractal_dimension']
column_names = ['id', 'diagnosis']

for attr in ['mean', 'ste', 'largest']:
  for feature in features:
    column_names.append(feature + "_" + attr)

[`pandas.read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html){:.external}를 사용하여 데이터세트를 pandas [DataFrame](){:.external}로 읽어옵니다.

In [ ]:
dataset = pd.read_csv(url, names=column_names)

In [ ]:
dataset.info()

처음 5개 행을 표시합니다.

In [ ]:
dataset.head()

[`pandas.DataFrame.sample`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html){:.external}, [`pandas.DataFrame.drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html){:.external}, [`pandas.DataFrame.iloc`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html){:.external}을 사용하여 데이터세트를 훈련 세트와 테스트 세트로 분할합니다. 대상 레이블에서 특성을 분할했는지 확인해야 합니다. 테스트 세트는 보이지 않는 데이터에서 모델의 일반화 가능성을 평가하는 데 사용합니다.

In [ ]:
train_dataset = dataset.sample(frac=0.75, random_state=1)

In [ ]:
len(train_dataset)

In [ ]:
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
len(test_dataset)

In [ ]:
# The `id` column can be dropped since each row is unique
x_train, y_train = train_dataset.iloc[:, 2:], train_dataset.iloc[:, 1]
x_test, y_test = test_dataset.iloc[:, 2:], test_dataset.iloc[:, 1]

## 데이터 전처리

이 데이터세트는 예제별로 수집한 10개의 종양 측정값 각각에 대한 평균, 표준 오차 및 최댓값을 포함하고 있습니다. `"diagnosis"` 대상 열은 악성 종양을 나타내는 `'M'`과 양성 종양 진단을 나타내는 `'B'`가 있는 범주형 변수입니다. 이 열은 모델 훈련을 위해 숫자 바이너리 형식으로 변환해야 합니다.

[`pandas.Series.map`](https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html){:.external} 함수는 바이너리 값을 범주에 매핑하는 경우에 유용합니다.

전처리를 완료한 후 데이터세트도 `tf.convert_to_tensor` 함수를 사용하여 텐서로 변환해야 합니다.

In [ ]:
y_train, y_test = y_train.map({'B': 0, 'M': 1}), y_test.map({'B': 0, 'M': 1})
x_train, y_train = tf.convert_to_tensor(x_train, dtype=tf.float32), tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test, y_test = tf.convert_to_tensor(x_test, dtype=tf.float32), tf.convert_to_tensor(y_test, dtype=tf.float32)

[`seaborn.pairplot`](https://seaborn.pydata.org/generated/seaborn.pairplot.html){:.external}을 사용하여 훈련 세트에서 몇 쌍의 평균 기반 특성의 결합 분포를 검토하고 대상과 어떤 관련이 있는지 관찰합니다.

In [ ]:
sns.pairplot(train_dataset.iloc[:, 1:6], hue = 'diagnosis', diag_kind='kde');

이 페어플롯은 반경, 둘레 및 면적과 같은 특정 특성이 높은 상관 관계가 있음을 보여줍니다. 이것은 종양 반경이 둘레와 면적의 계산 모두와 직접 관련되기 때문인 것으로 예상됩니다. 또한 악성 진단은 많은 특성에 대해 오른쪽으로 더 많이 치우친 것처럼 보입니다.

전체 통계도 확인해야 합니다. 각 특성이 매우 다양한 값의 범위를 어떻게 다루는지 주목하세요.

In [ ]:
train_dataset.describe().transpose()[:10]

일관성 없는 범위가 주어지는 경우 각 특성이 제로 평균과 단위 분산을 갖도록 데이터를 표준화하는 것이 좋습니다. 이러한 프로세스를 [정규화](https://developers.google.com/machine-learning/glossary#normalization){:.external}라고 합니다.

In [ ]:
class Normalize(tf.Module):
  def __init__(self, x):
    # Initialize the mean and standard deviation for normalization
    self.mean = tf.Variable(tf.math.reduce_mean(x, axis=0))
    self.std = tf.Variable(tf.math.reduce_std(x, axis=0))

  def norm(self, x):
    # Normalize the input
    return (x - self.mean)/self.std

  def unnorm(self, x):
    # Unnormalize the input
    return (x * self.std) + self.mean

norm_x = Normalize(x_train)
x_train_norm, x_test_norm = norm_x.norm(x_train), norm_x.norm(x_test)

## 로지스틱 회귀

로지스틱 회귀 모델을 구축하기 전에 기존 선형 회귀와 비교하며 차이점을 이해하는 것이 중요합니다.

### 로지스틱 회귀 기본 사항

선형 회귀는 입력 항목의 선형 조합을 반환합니다. 이 경우 출력의 제한이 없습니다. [로지스틱 회귀](https://developers.google.com/machine-learning/glossary#logistic_regression){:.external}의 출력은 `(0, 1)` 범위 내에서 이루어집니다.각 예제는 해당 예제가 *positive* 클래스에 속할 확률을 나타냅니다.

로지스틱 회귀는 기존 선형 회귀 `(-∞, ∞)`의 연속 출력을 확률 `(0, 1)`에 매핑합니다. 이 변환도 대칭적이기에 선형 출력의 부호를 뒤집으면 원래 확률의 반대가 됩니다.

$Y$는 클래스 `1`에 속할 확률을 나타냅니다(종양은 악성임). 선형 회귀 출력을 클래스 `0`이 아닌 클래스 `1`에 있는 [로그 오즈(log odds)](https://developers.google.com/machine-learning/glossary#log-odds){:.external} 비율로 해석하여 원하는 매핑 결과를 얻을 수 있습니다.

```
$wX + b = z$를 설정하면 $Y$에 대한 이 방정식을 풀 수 있습니다.
```

$\frac{1}{1 + e^{-z}}$ 표현식은 [시그모이드 함수](https://developers.google.com/machine-learning/glossary#sigmoid_function){:.external} $\sigma(z)$로 알려져 있습니다. 따라서 로지스틱 회귀 방정식은 $Y = \sigma(wX + b)$로 작성할 수 있습니다.

```
이 튜토리얼의 데이터세트는 고차원 특성 행렬을 처리합니다. 따라서 위의 수식은 다음과 같이 행렬 벡터 형식으로 다시 작성해야 합니다.
```

여기서,

먼저 선형 출력 `(-∞, ∞)`이 `0`과 `1` 사이에 있도록 변환하는 시그모이드 함수를 시각화합니다. 시그모이드 함수는 `tf.math.sigmoid`에서 사용할 수 있습니다.

```
$${\mathrm{Y}} = \sigma({\mathrm{X}}w + b)$$
```

여기서:

- $\underset{m\times 1}{\mathrm{Y}}$: 목표 벡터
- $\underset{m\times n}{\mathrm{X}}$: 특성 행렬
- $\underset{n\times 1}w$: 가중치 벡터
- $b$: 바이어스
- $\sigma$: 출력 벡터의 각 요소에 적용되는 시그모이드 함수

먼저 선형 출력 `(-∞, ∞)`이 `0`과 `1` 사이에 있도록 변환하는 시그모이드 함수를 시각화합니다. 시그모이드 함수는 `tf.math.sigmoid`에서 사용할 수 있습니다.

In [ ]:
x = tf.linspace(-10, 10, 500)
x = tf.cast(x, tf.float32)
f = lambda x : (1/20)*x + 0.6
plt.plot(x, tf.math.sigmoid(x))
plt.ylim((-0.1,1.1))
plt.title("Sigmoid function");

### 로그 손실 함수

[로그 손실](https://developers.google.com/machine-learning/glossary#Log_Loss){:.external} 또는 바이너리 교차 엔트로피 손실은 로지스틱 회귀가 있는 바이너리 분류 문제를 다루는 이상적인 손실 함수입니다. 각 예제에서 로그 손실은 예측 확률과 예제의 실제 값 사이의 유사성을 수량화합니다. 이는 다음 수식에 의해 결정됩니다.

```
여기서,
```

`tf.nn.sigmoid_cross_entropy_with_logits` 함수를 사용하여 로그 손실을 계산할 수 있습니다. 이 함수는 시그모이드 활성화를 회귀 출력에 자동으로 적용합니다.

- $\hat{y}$: 예측 확률의 벡터
- $y$: 실제 목표의 벡터

`tf.nn.sigmoid_cross_entropy_with_logits` 함수를 사용하여 로그 손실을 계산할 수 있습니다. 이 함수는 시그모이드 활성화를 회귀 출력에 자동으로 적용합니다.

In [ ]:
def log_loss(y_pred, y):
  # Compute the log loss function
  ce = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=y_pred)
  return tf.reduce_mean(ce)

### 경사 하강 업데이트 규칙

TensorFlow Core API는 `tf.GradientTape`를 사용하는 자동 미분을 지원합니다. 로지스틱 회귀 [그래디언트 업데이트](https://developers.google.com/machine-learning/glossary#gradient_descent){:.external} 이면의 수학에 대해 궁금한 경우 다음과 같은 간단한 설명을 참고해 주세요.

위의 로그 손실 수식에서 각 $\hat{y}_i$는 $\sigma({\mathrm{X_i}}w + b)$와 같이 입력에서 다시 작성할 수 있음을 기억해야 합니다.

목표는 로그 손실을 최소화하는 $w^*$와 $b^*$를 찾는 것입니다.

```
$w$에 대해 그래디언트 $L$를 선택하면 다음을 얻습니다.
```

$b$에 대해 그래디언트 $L$를 선택하면 다음을 얻습니다.

```
$$\frac{\partial L}{\partial w} = \frac{1}{m}(\sigma({\mathrm{X}}w + b) - y)X$$
```

$b$에 대해 그래디언트 $L$를 선택하면 다음을 얻습니다.

```
$$\frac{\partial L}{\partial b} = \frac{1}{m}\sum_{i=1}^{m}\sigma({\mathrm{X_i}}w + b) - y_i$$
```

이제 로지스틱 회귀 모델을 빌드합니다.

In [ ]:
class LogisticRegression(tf.Module):

  def __init__(self):
    self.built = False
    
  def __call__(self, x, train=True):
    # Initialize the model parameters on the first call
    if not self.built:
      # Randomly generate the weights and the bias term
      rand_w = tf.random.uniform(shape=[x.shape[-1], 1], seed=22)
      rand_b = tf.random.uniform(shape=[], seed=22)
      self.w = tf.Variable(rand_w)
      self.b = tf.Variable(rand_b)
      self.built = True
    # Compute the model output
    z = tf.add(tf.matmul(x, self.w), self.b)
    z = tf.squeeze(z, axis=1)
    if train:
      return z
    return tf.sigmoid(z)

검증하려면 훈련되지 않은 모델이 훈련 데이터의 작은 하위 집합에 대해 `(0, 1)` 범위의 값을 출력하는지 확인해야 합니다.

In [ ]:
log_reg = LogisticRegression()

In [ ]:
y_pred = log_reg(x_train_norm[:5], train=False)
y_pred.numpy()

다음으로, 훈련하는 동안 올바른 분류의 비율을 계산하는 정확성 함수를 작성합니다. 예측 확률에서 분류를 검색하려면 임계값보다 높은 모든 확률이 클래스 `1`에 속하는 임계값을 설정해야 합니다. 이는 기본값으로 `0.5`로 설정할 수 있는 구성 가능한 하이퍼 매개변수입니다.

In [ ]:
def predict_class(y_pred, thresh=0.5):
  # Return a tensor with  `1` if `y_pred` > `0.5`, and `0` otherwise
  return tf.cast(y_pred > thresh, tf.float32)

def accuracy(y_pred, y):
  # Return the proportion of matches between `y_pred` and `y`
  y_pred = tf.math.sigmoid(y_pred)
  y_pred_class = predict_class(y_pred)
  check_equal = tf.cast(y_pred_class == y,tf.float32)
  acc_val = tf.reduce_mean(check_equal)
  return acc_val

### 모델 훈련하기

훈련에 미니 배치를 사용하면 메모리 효율성이 높아지고 더 빠른 수렴이 가능해집니다. `tf.data.Dataset` API에는 배치와 셔플에 사용할 수 있는 유용한 함수가 있습니다. API를 사용하면 간단하고 재사용 가능한 부분으로부터 복잡한 입력 파이프라인을 빌드할 수 있습니다. 

In [ ]:
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_norm, y_train))
train_dataset = train_dataset.shuffle(buffer_size=x_train.shape[0]).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_norm, y_test))
test_dataset = test_dataset.shuffle(buffer_size=x_test.shape[0]).batch(batch_size)

이제 로지스틱 회귀 모델의 훈련 루프를 작성합니다. 루프는 모델의 매개변수를 반복적으로 업데이트하기 위해 입력에 대한 로그 손실 함수와 그래디언트를 활용합니다.

In [ ]:
# Set training parameters
epochs = 200
learning_rate = 0.01
train_losses, test_losses = [], []
train_accs, test_accs = [], []

# Set up the training loop and begin training
for epoch in range(epochs):
  batch_losses_train, batch_accs_train = [], []
  batch_losses_test, batch_accs_test = [], []

  # Iterate over the training data
  for x_batch, y_batch in train_dataset:
    with tf.GradientTape() as tape:
      y_pred_batch = log_reg(x_batch)
      batch_loss = log_loss(y_pred_batch, y_batch)
    batch_acc = accuracy(y_pred_batch, y_batch)
    # Update the parameters with respect to the gradient calculations
    grads = tape.gradient(batch_loss, log_reg.variables)
    for g,v in zip(grads, log_reg.variables):
      v.assign_sub(learning_rate * g)
    # Keep track of batch-level training performance
    batch_losses_train.append(batch_loss)
    batch_accs_train.append(batch_acc)

  # Iterate over the testing data
  for x_batch, y_batch in test_dataset:
    y_pred_batch = log_reg(x_batch)
    batch_loss = log_loss(y_pred_batch, y_batch)
    batch_acc = accuracy(y_pred_batch, y_batch)
    # Keep track of batch-level testing performance
    batch_losses_test.append(batch_loss)
    batch_accs_test.append(batch_acc)

  # Keep track of epoch-level model performance
  train_loss, train_acc = tf.reduce_mean(batch_losses_train), tf.reduce_mean(batch_accs_train)
  test_loss, test_acc = tf.reduce_mean(batch_losses_test), tf.reduce_mean(batch_accs_test)
  train_losses.append(train_loss)
  train_accs.append(train_acc)
  test_losses.append(test_loss)
  test_accs.append(test_acc)
  if epoch % 20 == 0:
    print(f"Epoch: {epoch}, Training log loss: {train_loss:.3f}")

### 성능 평가

시간 경과에 따른 모델 손실 및 정확성의 변화를 관찰합니다. 

In [ ]:
plt.plot(range(epochs), train_losses, label = "Training loss")
plt.plot(range(epochs), test_losses, label = "Testing loss")
plt.xlabel("Epoch")
plt.ylabel("Log loss")
plt.legend()
plt.title("Log loss vs training iterations");

In [ ]:
plt.plot(range(epochs), train_accs, label = "Training accuracy")
plt.plot(range(epochs), test_accs, label = "Testing accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.title("Accuracy vs training iterations");

In [ ]:
print(f"Final training log loss: {train_losses[-1]:.3f}")
print(f"Final testing log Loss: {test_losses[-1]:.3f}")

In [ ]:
print(f"Final training accuracy: {train_accs[-1]:.3f}")
print(f"Final testing accuracy: {test_accs[-1]:.3f}")

이 모델은 훈련 데이터세트에서 종양을 분류할 경우 높은 정확성과 낮은 손실을 보여주며 보이지 않는 테스트 데이터에도 잘 일반화됩니다. 한 단계 더 나아가 전체 정확성 점수보다 더 많은 인사이트를 제공하는 오류율을 탐색할 수도 있습니다. 바이너리 분류 문제에서 가장 많이 사용되는 두 가지 오류율은 FPR(거짓양성률)과 FNR(거짓음성률)입니다.

이 문제에서 FPR은 실제로 양성인 종양에서 악성 종양 예측의 비율입니다. 반대로, FNR은 실제로 악성인 종양에서 양성 종양 예측의 비율입니다.

분류의 정확성을 평가하는 [`sklearn.metrics.confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix){:.external}를 사용하여 오차 행렬을 계산하고 matplotlib를 사용하여 행렬을 표시합니다.

In [ ]:
def show_confusion_matrix(y, y_classes, typ):
  # Compute the confusion matrix and normalize it
  plt.figure(figsize=(10,10))
  confusion = sk_metrics.confusion_matrix(y.numpy(), y_classes.numpy())
  confusion_normalized = confusion / confusion.sum(axis=1, keepdims=True)
  axis_labels = range(2)
  ax = sns.heatmap(
      confusion_normalized, xticklabels=axis_labels, yticklabels=axis_labels,
      cmap='Blues', annot=True, fmt='.4f', square=True)
  plt.title(f"Confusion matrix: {typ}")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

y_pred_train, y_pred_test = log_reg(x_train_norm, train=False), log_reg(x_test_norm, train=False)
train_classes, test_classes = predict_class(y_pred_train), predict_class(y_pred_test)

In [ ]:
show_confusion_matrix(y_train, train_classes, 'Training')

In [ ]:
show_confusion_matrix(y_test, test_classes, 'Testing')

이 예제의 맥락에서 오류율 측정을 관찰하고 그 중요성을 해석합니다. 암 진단과 같은 많은 의료 테스트 연구에서 낮은 거짓음성률을 보장하기 위해 높은 거짓양성률을 갖는 것은 완벽하게 수용 가능하며 실제로 권장되는 이유는 악성 종양 진단(거짓 음성)을 놓치는 위험이 양성 종양을 악성으로 잘못 분류하는 것(거짓 양성)보다 훨씬 낫기 때문입니다

FPR 및 FNR을 제어하려면 확률 예측을 분류하기 전에 임계값 하이퍼 매개변수를 변경해 봅니다. 임계값이 낮을수록 모델의 악성 종양 분류 가능성이 전반적으로 높아집니다. 이는 필연적으로 거짓 양성과 FPR의 수를 증가시키지만 거짓 음성 및 FNR의 수를 줄이는 데에도 도움이 됩니다.

## 모델 저장하기

먼저 원시 데이터를 선택하고 다음 연산을 수행하는 내보내기 모듈을 만들어 봅니다.

- 정규화
- 확률 예측
- 클래스 예측


In [ ]:
class ExportModule(tf.Module):
  def __init__(self, model, norm_x, class_pred):
    # Initialize pre- and post-processing functions
    self.model = model
    self.norm_x = norm_x
    self.class_pred = class_pred

  @tf.function(input_signature=[tf.TensorSpec(shape=[None, None], dtype=tf.float32)])
  def __call__(self, x):
    # Run the `ExportModule` for new data points
    x = self.norm_x.norm(x)
    y = self.model(x, train=False)
    y = self.class_pred(y)
    return y 

In [ ]:
log_reg_export = ExportModule(model=log_reg,
                              norm_x=norm_x,
                              class_pred=predict_class)

현재 상태로 모델을 저장하기 위해 `tf.saved_model.save` 함수를 사용할 수 있습니다. 저장한 모델을 로드하고 예측하려면 `tf.saved_model.load` 함수를 사용합니다.

In [ ]:
models = tempfile.mkdtemp()
save_path = os.path.join(models, 'log_reg_export')
tf.saved_model.save(log_reg_export, save_path)

In [ ]:
log_reg_loaded = tf.saved_model.load(save_path)
test_preds = log_reg_loaded(x_test)
test_preds[:10].numpy()

## 결론

이 노트북에서는 로지스틱 회귀 문제를 처리하는 몇 가지 기술을 소개했습니다. 다음은 도움이 될 수 있는 몇 가지 추가 팁입니다.

- 구성 가능성이 높은 머신러닝 워크플로를 구축하는 데 [TensorFlow Core API](https://www.tensorflow.org/guide/core)를 사용할 수 있습니다.
- 오류율 분석은 전체 정확성 점수 이면의 분류 모델의 성능에 대한 더 많은 인사이트를 얻을 수 있는 좋은 방법입니다.
- 과대적합은 로지스틱 회귀 모델의 또 다른 일반적인 문제이지만 이 튜토리얼에서는 문제가 되지 않았습니다. 이에 대한 추가적인 도움이 필요하면 [과대적합 및 과소적합](../../tutorials/keras/overfit_and_underfit.ipynb) 튜토리얼을 참조하세요.

TensorFlow Core API를 사용하는 더 많은 예제는 [가이드](https://www.tensorflow.org/guide/core)를 확인하세요. 데이터 로드 및 준비에 대해 자세히 알아보려면 [이미지 데이터 로드](../../tutorials/load_data/images.ipynb) 또는 [CSV 데이터 로드](../../tutorials/load_data/csv.ipynb) 튜토리얼을 참고하세요.